Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 1. Create Label Map

In [3]:
labels = [{'name':'hello', 'id':1}, 
          {'name':'thanks', 'id':2},
          {'name':'please', 'id':3}, 
          {'name':'sorry', 'id':4},
          {'name':'yes', 'id':5}, 
          {'name':'no', 'id':6},
          {'name':'help', 'id':7},
         ]

# Use os.path.join() for cross-platform compatibility
import os
with open(os.path.join(ANNOTATION_PATH, 'label_map.pbtxt'), 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [7]:
import os
!python {os.path.join(SCRIPTS_PATH, 'generate_tfrecord.py')} -x {os.path.join(IMAGE_PATH, 'train')} -l {os.path.join(ANNOTATION_PATH, 'label_map.pbtxt')} -o {os.path.join(ANNOTATION_PATH, 'train.record')}
!python {os.path.join(SCRIPTS_PATH, 'generate_tfrecord.py')} -x {os.path.join(IMAGE_PATH, 'test')} -l {os.path.join(ANNOTATION_PATH, 'label_map.pbtxt')} -o {os.path.join(ANNOTATION_PATH, 'test.record')}

Label map dictionary: {'hello': 2, 'thanks': 3, 'please': 4, 'sorry': 5, 'yes': 6, 'no': 7}
Found 84 annotations
Traceback (most recent call last):
  File "/Users/prithvibaddi/RealTimeObjectDetection/Tensorflow/scripts/generate_tfrecord.py", line 127, in <module>
    tf.compat.v1.app.run()
  File "/opt/anaconda3/envs/tfod/lib/python3.9/site-packages/tensorflow/python/platform/app.py", line 36, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/opt/anaconda3/envs/tfod/lib/python3.9/site-packages/absl/app.py", line 316, in run
    _run_main(main, args)
  File "/opt/anaconda3/envs/tfod/lib/python3.9/site-packages/absl/app.py", line 261, in _run_main
    sys.exit(main(argv))
  File "/Users/prithvibaddi/RealTimeObjectDetection/Tensorflow/scripts/generate_tfrecord.py", line 119, in main
    tf_example = create_tf_example(group.iloc[0], FLAGS.x, label_map_dict)
  File "/Users/prithvibaddi/RealTimeObjectDetection/Tensorflow/scripts/generate_tfrecord.py", l

# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [4]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 102330, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 102330 (delta 10), reused 8 (delta 8), pack-reused 102315 (from 2)
Receiving objects: 100% (102330/102330), 642.70 MiB | 11.40 MiB/s, done.
Resolving deltas: 100% (73895/73895), done.
Updating files: 100% (3933/3933), done.


In [6]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [26]:
import os
import shutil

# Remove the incorrect file first
target_path = f'Tensorflow/workspace/models/{CUSTOM_MODEL_NAME}'
if os.path.exists(target_path):
    if os.path.isfile(target_path):
        print(f"Removing incorrect file: {target_path}")
        os.remove(target_path)

# Create the directory properly
os.makedirs(target_path, exist_ok=True)
print(f"Created directory: {target_path}")

# Copy the config file
source = f'{PRETRAINED_MODEL_PATH}/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'
destination = f'{target_path}/pipeline.config'
shutil.copy(source, destination)
print(f"Copied pipeline.config to: {destination}")

# Verify it worked
if os.path.isdir(target_path) and os.path.exists(destination):
    print("✅ Success! Directory created and config file copied.")
    print(f"Contents: {os.listdir(target_path)}")
else:
    print("❌ Something went wrong")

Removing incorrect file: Tensorflow/workspace/models/my_ssd_mobnet
Created directory: Tensorflow/workspace/models/my_ssd_mobnet
Copied pipeline.config to: Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config
✅ Success! Directory created and config file copied.
Contents: ['pipeline.config']


# 5. Update Config For Transfer Learning

In [1]:
# Test script
import sys

try:
    import tensorflow as tf
    print(f"✅ TensorFlow {tf.__version__} imported successfully!")
    
    # Add Object Detection API path
    research_path = "/Users/prithvibaddi/RealTimeObjectDetection/models/research"
    if research_path not in sys.path:
        sys.path.insert(0, research_path)
    
    from object_detection.utils import config_util
    from object_detection.protos import pipeline_pb2
    from google.protobuf import text_format
    print("✅ Object Detection API imported successfully!")
    
except Exception as e:
    print(f"❌ Error: {e}")

✅ TensorFlow 2.13.0 imported successfully!
✅ Object Detection API imported successfully!


In [2]:
import sys
import site

print("Python executable:", sys.executable)
print("Python path:")
for path in sys.path:
    print(f"  {path}")
print("\nSite packages:")
for path in site.getsitepackages():
    print(f"  {path}")

Python executable: /opt/anaconda3/envs/tfod/bin/python
Python path:
  /opt/anaconda3/envs/tfod/lib/python39.zip
  /opt/anaconda3/envs/tfod/lib/python3.9
  /opt/anaconda3/envs/tfod/lib/python3.9/lib-dynload
  
  /opt/anaconda3/envs/tfod/lib/python3.9/site-packages
  /opt/anaconda3/envs/tfod/lib/python3.9/site-packages/setuptools/_vendor

Site packages:
  /opt/anaconda3/envs/tfod/lib/python3.9/site-packages


In [3]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [4]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [5]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [56]:
config

{'model': ssd {
   num_classes: 2
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 4e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.01
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
    

In [8]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [31]:
pipeline_config.model.ssd.num_classes = 7
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [32]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [6]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=20000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=20000


# 7. Load Train Model From Checkpoint

In [10]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [19]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder
import os

# Your paths
CONFIG_PATH = 'Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config'  # Using the config from your checkpoint dir
CHECKPOINT_PATH = 'Tensorflow/workspace/models/my_ssd_mobnet'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Method 1: Use the latest checkpoint automatically (recommended)
latest_checkpoint = tf.train.latest_checkpoint(CHECKPOINT_PATH)
print(f"Latest checkpoint found: {latest_checkpoint}")

if latest_checkpoint:
    ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
    ckpt.restore(latest_checkpoint).expect_partial()
    print("✅ Latest checkpoint restored successfully!")
else:
    print("❌ No checkpoint found!")

# Method 2: If you want to use a specific checkpoint (e.g., ckpt-21)
# ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
# ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-21')).expect_partial()
# print("✅ Checkpoint ckpt-21 restored successfully!")

@tf.function
def detect_fn(image):
    """
    Detect objects in image.
    
    Args:
        image: A three-dimensional uint8 tensor of shape [height, width, 3]
        
    Returns:
        A dictionary containing detection results
    """
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

print("🎉 Model loaded and ready for inference!")

Latest checkpoint found: Tensorflow/workspace/models/my_ssd_mobnet/ckpt-21
✅ Latest checkpoint restored successfully!
🎉 Model loaded and ready for inference!


In [21]:
import os

# Check your training data balance
TRAIN_PATH = os.path.join(IMAGE_PATH, 'train')
TEST_PATH = os.path.join(IMAGE_PATH, 'test')

labels = ['Hello', 'Thanks', 'Please', 'Sorry', 'Yes', 'No', 'Help']

print("Training data distribution:")
for label in labels:
    train_count = len([f for f in os.listdir(TRAIN_PATH) if f.startswith(label.lower()) and f.endswith('.jpg')])
    test_count = len([f for f in os.listdir(TEST_PATH) if f.startswith(label.lower()) and f.endswith('.jpg')])
    xml_train = len([f for f in os.listdir(TRAIN_PATH) if f.startswith(label.lower()) and f.endswith('.xml')])
    xml_test = len([f for f in os.listdir(TEST_PATH) if f.startswith(label.lower()) and f.endswith('.xml')])
    
    print(f"{label}: Train={train_count} imgs, {xml_train} annotations | Test={test_count} imgs, {xml_test} annotations")

Training data distribution:
Hello: Train=0 imgs, 0 annotations | Test=0 imgs, 0 annotations
Thanks: Train=0 imgs, 0 annotations | Test=0 imgs, 0 annotations
Please: Train=0 imgs, 0 annotations | Test=0 imgs, 0 annotations
Sorry: Train=0 imgs, 0 annotations | Test=0 imgs, 0 annotations
Yes: Train=0 imgs, 0 annotations | Test=0 imgs, 0 annotations
No: Train=0 imgs, 0 annotations | Test=0 imgs, 0 annotations
Help: Train=0 imgs, 0 annotations | Test=0 imgs, 0 annotations


In [22]:
import os

IMAGE_PATH = 'Tensorflow/workspace/images'  # Update this to your actual path
TRAIN_PATH = os.path.join(IMAGE_PATH, 'train')
TEST_PATH = os.path.join(IMAGE_PATH, 'test')

# Check if directories exist
print(f"IMAGE_PATH exists: {os.path.exists(IMAGE_PATH)}")
print(f"TRAIN_PATH exists: {os.path.exists(TRAIN_PATH)}")
print(f"TEST_PATH exists: {os.path.exists(TEST_PATH)}")

if os.path.exists(TRAIN_PATH):
    train_files = os.listdir(TRAIN_PATH)
    print(f"Files in train directory: {len(train_files)}")
    print(f"First 10 files: {train_files[:10]}")

if os.path.exists(TEST_PATH):
    test_files = os.listdir(TEST_PATH)
    print(f"Files in test directory: {len(test_files)}")
    print(f"First 10 files: {test_files[:10]}")

IMAGE_PATH exists: True
TRAIN_PATH exists: True
TEST_PATH exists: True
Files in train directory: 170
First 10 files: ['Sorry.77e66eac-853c-11f0-acb2-1a80c522e9da.jpg', 'Please.6d49aff4-853c-11f0-acb2-1a80c522e9da.jpg', 'Hello.3cde00e0-853c-11f0-acb2-1a80c522e9da.jpg', 'Please.6d49aff4-853c-11f0-acb2-1a80c522e9da.xml', 'Hello.3cde00e0-853c-11f0-acb2-1a80c522e9da.xml', 'Sorry.77e66eac-853c-11f0-acb2-1a80c522e9da.xml', 'Sorry.7e1b0b20-853c-11f0-acb2-1a80c522e9da.jpg', 'Sorry.7e1b0b20-853c-11f0-acb2-1a80c522e9da.xml', 'No.a713e934-853c-11f0-acb2-1a80c522e9da.xml', 'No.a713e934-853c-11f0-acb2-1a80c522e9da.jpg']
Files in test directory: 44
First 10 files: ['Help.c090c8aa-853c-11f0-acb2-1a80c522e9da.jpg', 'Please.69937160-853c-11f0-acb2-1a80c522e9da.jpg', 'Help.c090c8aa-853c-11f0-acb2-1a80c522e9da.xml', 'Please.69937160-853c-11f0-acb2-1a80c522e9da.xml', 'No.ac063da2-853c-11f0-acb2-1a80c522e9da.jpg', 'No.ac063da2-853c-11f0-acb2-1a80c522e9da.xml', 'Thanks.46363752-853c-11f0-acb2-1a80c522e9da.xm

In [23]:
# Search for your actual training data
import glob

# Look for image files in your workspace
image_patterns = ['**/*.jpg', '**/*.jpeg', '**/*.png']
xml_patterns = ['**/*.xml']

print("Searching for image files...")
for pattern in image_patterns:
    files = glob.glob(f'Tensorflow/workspace/{pattern}', recursive=True)
    if files:
        print(f"Found {len(files)} images matching {pattern}")
        for f in files[:5]:  # Show first 5
            print(f"  {f}")

print("\nSearching for annotation files...")
for pattern in xml_patterns:
    files = glob.glob(f'Tensorflow/workspace/{pattern}', recursive=True)
    if files:
        print(f"Found {len(files)} XML files matching {pattern}")
        for f in files[:5]:  # Show first 5
            print(f"  {f}")

Searching for image files...
Found 105 images matching **/*.jpg
  Tensorflow/workspace/images/test/Help.c090c8aa-853c-11f0-acb2-1a80c522e9da.jpg
  Tensorflow/workspace/images/test/Please.69937160-853c-11f0-acb2-1a80c522e9da.jpg
  Tensorflow/workspace/images/test/No.ac063da2-853c-11f0-acb2-1a80c522e9da.jpg
  Tensorflow/workspace/images/test/Thanks.46363752-853c-11f0-acb2-1a80c522e9da.jpg
  Tensorflow/workspace/images/test/Thanks.57881444-853c-11f0-acb2-1a80c522e9da.jpg

Searching for annotation files...
Found 105 XML files matching **/*.xml
  Tensorflow/workspace/images/test/Help.c090c8aa-853c-11f0-acb2-1a80c522e9da.xml
  Tensorflow/workspace/images/test/Please.69937160-853c-11f0-acb2-1a80c522e9da.xml
  Tensorflow/workspace/images/test/No.ac063da2-853c-11f0-acb2-1a80c522e9da.xml
  Tensorflow/workspace/images/test/Thanks.46363752-853c-11f0-acb2-1a80c522e9da.xml
  Tensorflow/workspace/images/test/Thanks.57881444-853c-11f0-acb2-1a80c522e9da.xml


In [ ]:
import os
import time
from datetime import datetime

checkpoint_dir = 'Tensorflow/workspace/models/my_ssd_mobnet'

print("Monitoring training progress...")
print("Press Ctrl+C to stop monitoring")

try:
    while True:
        checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith('ckpt-') and f.endswith('.index')]
        
        if checkpoint_files:
            checkpoint_numbers = [int(f.split('-')[1].split('.')[0]) for f in checkpoint_files]
            latest_num = max(checkpoint_numbers)
            
            latest_file = f'ckpt-{latest_num}.index'
            latest_path = os.path.join(checkpoint_dir, latest_file)
            
            if os.path.exists(latest_path):
                mod_time = os.path.getmtime(latest_path)
                readable_time = datetime.fromtimestamp(mod_time).strftime('%H:%M:%S')
                current_time = datetime.now().strftime('%H:%M:%S')
                
                time_since_update = (time.time() - mod_time) / 60  # minutes
                
                print(f"[{current_time}] Checkpoint: ckpt-{latest_num} | Last update: {readable_time} ({time_since_update:.1f}m ago)")
                
                if time_since_update > 15:  # Alert if no update for 15 minutes
                    print("  WARNING: No updates for >15 minutes - training may be stalled")
        
        time.sleep(60)  # Check every minute
        
except KeyboardInterrupt:
    print("\nMonitoring stopped.")


Monitoring training progress...
Press Ctrl+C to stop monitoring
[21:36:51] Checkpoint: ckpt-22 | Last update: 21:36:29 (0.4m ago)
[21:37:51] Checkpoint: ckpt-24 | Last update: 21:37:39 (0.2m ago)
[21:38:51] Checkpoint: ckpt-26 | Last update: 21:38:40 (0.2m ago)
[21:39:51] Checkpoint: ckpt-28 | Last update: 21:39:45 (0.1m ago)
[21:40:51] Checkpoint: ckpt-30 | Last update: 21:40:49 (0.0m ago)
[21:41:51] Checkpoint: ckpt-31 | Last update: 21:41:21 (0.5m ago)
[21:42:51] Checkpoint: ckpt-33 | Last update: 21:42:27 (0.4m ago)
[21:43:51] Checkpoint: ckpt-35 | Last update: 21:43:37 (0.2m ago)
[21:44:51] Checkpoint: ckpt-37 | Last update: 21:44:46 (0.1m ago)
[21:45:51] Checkpoint: ckpt-38 | Last update: 21:45:21 (0.5m ago)
[21:46:51] Checkpoint: ckpt-40 | Last update: 21:46:31 (0.3m ago)
[21:47:51] Checkpoint: ckpt-42 | Last update: 21:47:39 (0.2m ago)
[21:48:51] Checkpoint: ckpt-44 | Last update: 21:48:51 (0.0m ago)
[21:49:51] Checkpoint: ckpt-45 | Last update: 21:49:26 (0.4m ago)
[21:50:51] C

# 8. Detect in Real-Time

In [9]:
import cv2 
import numpy as np

In [10]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [11]:
# Setup capture
cap = cv2.VideoCapture(1, cv2.CAP_AVFOUNDATION)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
        
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    # Debug: Print detection info
    print(f"Detections found: {num_detections}")
    if num_detections > 0:
        print(f"Top detection scores: {detections['detection_scores'][:3]}")
        print(f"Top detection classes: {detections['detection_classes'][:3]}")
    
    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.3,  # Lowered from .5 to .3
                agnostic_mode=False,
                # Improved text visibility
                line_thickness=3,
                groundtruth_box_visualization_color='black',
                skip_scores=False,  # Show confidence scores
                skip_labels=False)  # Show labels
    
    # Make window larger for better visibility
    resized_image = cv2.resize(image_np_with_detections, (1200, 800))  # Increased size
    cv2.imshow('Sign Language Detection', resized_image)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

2025-08-30 21:54:20.868519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Detections found: 100
Top detection scores: [0.69053113 0.17700973 0.1284561 ]
Top detection classes: [0 0 0]
Detections found: 100
Top detection scores: [0.7884254  0.7633527  0.16464444]
Top detection classes: [0 0 0]
Detections found: 100
Top detection scores: [0.8014289  0.65962994 0.15662841]
Top detection classes: [0 0 0]
Detections found: 100
Top detection scores: [0.81752735 0.8102242  0.17878921]
Top detection classes: [0 0 0]
Detections found: 100
Top detection scores: [0.8195486  0.75396645 0.17901888]
Top detection classes: [0 0 0]
Detections found: 100
Top detection scores: [0.82654387 0.7790329  0.18357436]
Top detection classes: [0 0 0]
Detections found: 100
Top detection scores: [0.8141876  0.80749613 0.19077806]
Top detection classes: [0 0 0]
Detections found: 100
Top detection scores: [0.82012033 0.80090314 0.1972525 ]
Top detection classes: [0 0 0]
Detections found: 100
Top detection scores: [0.8059341  0.6815567  0.16158074]
Top detection classes: [0 0 0]
Detections

In [42]:
detections = detect_fn(input_tensor)

In [67]:
from matplotlib import pyplot as plt